Membres du groupe 
- AFOMALE David
- DOGBO Sarah
- BOTRE Aboudou
- TEPE Paulin
- NOYOULIWA Victoire

### Contexte général
Préparation des données ASR en langue Ewe

Ce notebook décrit et exécute l'ensemble des étapes nécessaires
pour transformer les données brutes collectées (audio + texte)
en un dataset utilisable pour l'entraînement ou l'évaluation
d'un modèle de reconnaissance automatique de la parole (ASR).




In [ ]:
from pathlib import Path


**Vérification de la structure du projet s’assurer que les données RAW existent avant processing.**

In [ ]:
from src.config.settings import AUDIO_DIR, TEXT_DIR, META_DIR

print("Audio RAW :", len(list(AUDIO_DIR.glob("*.mp3"))))
print("Textes RAW :", len(list(TEXT_DIR.glob("*.txt"))))
print("Métadonnées :", META_DIR.exists())


Cette vérification permet de garantir que les données sources sont bien disponibles avant toute transformation.

### Étape 1 : Processing audio

Objectif méthodologique

Standardiser tous les fichiers audio au format requis par les modèles ASR (WAV, mono, 16 kHz).

In [ ]:
from src.preprocessing.audio_processing import convert_mp3_to_wav_16k

convert_mp3_to_wav_16k()


Explication :

- Suppression des variations de fréquence d’échantillonnage

- Format compatible Whisper / Wav2Vec2

**Contrôle qualité audio**

In [ ]:
from src.config.settings import PROJECT_ROOT

processed_audio = PROJECT_ROOT / "data" / "processed" / "audio_16k"
len(list(processed_audio.glob("*.wav")))


### Étape 2 : Nettoyage des transcriptions

**Objectif méthodologique**

Supprimer le bruit textuel (numéros de versets, espaces, caractères parasites).

In [ ]:
from src.preprocessing.text_cleaning import clean_all_texts

clean_all_texts()


**Exemple avant / après nettoyage**

In [ ]:
from src.config.settings import TEXT_DIR

raw_example = next(TEXT_DIR.glob("*.txt"))
print("TEXTE BRUT:\n", raw_example.read_text(encoding="utf-8")[:300])


In [ ]:
processed_text_dir = PROJECT_ROOT / "data" / "processed" / "transcripts"
cleaned_example = next(processed_text_dir.glob("*.txt"))
print("TEXTE NETTOYÉ:\n", cleaned_example.read_text(encoding="utf-8")[:300])

**Analyse :**

Le nettoyage améliore la qualité linguistique des transcriptions, ce qui est essentiel pour l’apprentissage ASR.

### Étape 3 : Construction du dataset ASR

**Objectif méthodologique**

Associer chaque fichier audio à sa transcription correspondante.

In [ ]:
from src.preprocessing.dataset_builder import build_asr_dataset

build_asr_dataset()


### ---------------------------------------------------------------------------

### Inspection du dataset final

In [ ]:
import pandas as pd

dataset_path = PROJECT_ROOT / "data" / "processed" / "ewe_asr_dataset.csv"
df = pd.read_csv(dataset_path)

df.head()


### Statistiques descriptives (méthodologie)

In [ ]:
df["duration_sec"] = df["audio_filepath"].apply(
    lambda p: Path(p)
)

len(df)


**Interpretation :**

- Nombre total d’exemples ASR
- Diversité textuelle
- Base suffisante pour évaluation / pré-entraînement